<a href="https://colab.research.google.com/github/DataNarrator/Datasets/blob/main/SMM_FLAGS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the R bridge and core R packages you likely need
!pip -q install rpy2==3.5.11

# (Optional) Faster CRAN mirror
R_CRAN = "https://cloud.r-project.org"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# Enable R cell magic
%load_ext rpy2.ipython

In [ ]:
%%R -i R_CRAN
# Install the R packages your Rmd uses
options(repos = c(CRAN = R_CRAN))
install.packages(c("tidyverse","readxl","janitor","rmarkdown","knitr"), quiet = TRUE)

In [8]:
from google.colab import files
uploaded = files.upload()


Saving mm_sample_input_long.csv to mm_sample_input_long.csv


In [9]:
from google.colab import files
uploaded = files.upload()


Saving smm_reference.csv to smm_reference.csv


In [32]:
!pip -q install pandas pyarrow openpyxl

import pandas as pd
import numpy as np
import re as re
import io as io
import glob, os

print("CWD:", os.getcwd())
print("Here:", sorted(os.listdir()))
print("CSVs:", glob.glob("*.csv"))
print("XLSX:", glob.glob("*.xlsx"))

CWD: /content
Here: ['.config', 'C:', 'mm_sample_input_long.csv', 'sample_data', 'smm_reference.csv']
CSVs: ['mm_sample_input_long.csv', 'smm_reference.csv']
XLSX: []


In [34]:
from google.colab import files
uploaded = files.upload()   # choose mm_sample_input_long.csv (and/or the ref, if needed)
print("Uploaded keys:", list(uploaded.keys()))

Saving mm_sample_input_long.csv to mm_sample_input_long.csv
Uploaded keys: ['mm_sample_input_long.csv']


In [35]:
import pandas as pd, io, csv

def read_csv_robust(path_or_bytes, prefer_sep=",", encodings=("utf-8-sig","cp1252","latin1")):
    """
    Reads CSV from a filename or bytes buffer. Tries multiple encodings and delimiters.
    Also normalizes non-breaking spaces (0xA0) in the raw stream.
    """
    # Get raw bytes
    if isinstance(path_or_bytes, (bytes, bytearray)):
        raw = bytes(path_or_bytes)
    elif isinstance(path_or_bytes, io.BytesIO):
        raw = path_or_bytes.getvalue()
    else:
        with open(path_or_bytes, "rb") as f:
            raw = f.read()

    # Replace NBSP with regular space before decoding
    raw = raw.replace(b"\xa0", b" ")
    # Try encodings
    last_err = None
    for enc in encodings:
        try:
            text = raw.decode(enc, errors="strict")
            break
        except UnicodeDecodeError as e:
            last_err = e
            text = None
    if text is None:  # fallback: replace errors
        text = raw.decode(encodings[0], errors="replace")

    # Delimiter sniff (fallback to provided prefer_sep)
    try:
        sniffer = csv.Sniffer()
        dialect = sniffer.sniff(text.splitlines(True)[0:1000][0] if text else "")
        sep = dialect.delimiter
    except Exception:
        # try common delims quickly
        for sep_try in (prefer_sep, "|", "\t", ";"):
            try:
                df = pd.read_csv(io.StringIO(text), sep=sep_try, dtype=str)
                if df.shape[1] > 1:
                    return df.fillna("")
            except Exception:
                pass
        # final attempt: default parser
        return pd.read_csv(io.StringIO(text), dtype=str).fillna("")

    # Use sniffed sep
    return pd.read_csv(io.StringIO(text), sep=sep, dtype=str).fillna("")

In [36]:
import io

# If you just uploaded, prefer those bytes; otherwise read from disk
def load_df(name):
    if 'uploaded' in globals() and name in uploaded:
        buf = uploaded[name]
        return read_csv_robust(buf)
    else:
        return read_csv_robust(name)

# Adjust names to match what you actually have
MM_FILE = "mm_sample_input_long.csv"   # change if your filename differs
REF_FILE = "smm_reference.csv"         # change if yours differs (or is .xlsx; see below)

# If your reference is actually Excel, read it like this instead:
# smm_ref = pd.read_excel(REF_FILE, dtype=str).fillna("")

mm_raw  = load_df(MM_FILE)
smm_ref = load_df(REF_FILE)

print("mm_raw shape:", mm_raw.shape)
print("smm_ref shape:", smm_ref.shape)
print("mm_raw columns head:", mm_raw.columns[:10].tolist())
print("smm_ref columns head:", smm_ref.columns[:10].tolist())

mm_raw shape: (144, 31)
smm_ref shape: (407, 8)
mm_raw columns head: ['DIM_MEMBER_KEY', 'ELIGIBILITY_BEGIN_DATE', 'ELIGIBILITY_END_DATE', 'ELIGIBILITY_BENEFIT_PLAN', 'MEMBER_AGE', 'MEMBER_RACE', 'MEMBER_ZIP_CODE_4', 'TEN_MONTHS_PRE_DELIVERY_DATE', 'DELIVERY_DATE', 'SIX_MONTHS_POST_DELIVERY_DATE']
smm_ref columns head: ['Flag#', 'Severe_Maternal_Morbidity_Indicator', 'Diagnosis_or_Procedure', 'ICD-10', 'Definition', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7']


In [38]:
print("smm_ref columns:", list(smm_ref.columns))
smm_ref.head(3)

smm_ref columns: ['FLAG#', 'SEVERE_MATERNAL_MORBIDITY_INDICATOR', 'DIAGNOSIS_OR_PROCEDURE', 'ICD-10', 'DEFINITION', 'UNNAMED: 5', 'UNNAMED: 6', 'UNNAMED: 7']


,FLAG#,SEVERE_MATERNAL_MORBIDITY_INDICATOR,DIAGNOSIS_OR_PROCEDURE,ICD-10,DEFINITION,UNNAMED: 5,UNNAMED: 6,UNNAMED: 7
0,1,Acute myocardial infarction,DX,I21.01,ST elevation (STEMI) myocardial infarction inv...,,,
1,1,Acute myocardial infarction,DX,I21.02,ST elevation (STEMI) myocardial infarction inv...,,,
2,1,Acute myocardial infarction,DX,I21.09,ST elevation (STEMI) myocardial infarction inv...,,,


In [39]:
import re

def normalize_name(name: str) -> str:
    """Uppercase and remove all non-alphanumerics for easy matching."""
    return re.sub(r'[^A-Z0-9]+', '', str(name).upper())

def guess_col(df, candidates):
    """Return the first df column whose normalized name matches any candidate (normalized)."""
    norm_map = {normalize_name(c): c for c in df.columns}
    for cand in candidates:
        key = normalize_name(cand)
        for col_norm, col_orig in norm_map.items():
            if col_norm == key:
                return col_orig
    # also try substring containment (looser)
    for cand in candidates:
        key = normalize_name(cand)
        for col_norm, col_orig in norm_map.items():
            if key in col_norm:
                return col_orig
    return None

# Common variants seen in public SMM ref files
diag_candidates = [
    "DIAGNOSIS", "DIAGNOSIS_OR_PROCEDURE", "Diagnosis or Procedure",
    "DX/PROC", "TYPE", "Diag_or_Proc"
]
icd_candidates = [
    "ICD_10", "ICD.10", "ICD-10", "ICD10", "ICD CODE", "CODE10", "CODE"
]
smm_candidates = [
    "SMM_INDICATOR", "Severe Maternal Morbidity Indicator",
    "SEVERE_MATERNAL_MORBIDITY_INDICATOR", "SMM INDICATOR", "INDICATOR"
]

diag_col = guess_col(smm_ref, diag_candidates)
icd_col  = guess_col(smm_ref, icd_candidates)
smm_col  = guess_col(smm_ref, smm_candidates)

print("Guessed columns →", "DIAGNOSIS:", diag_col, "| ICD_10:", icd_col, "| SMM_INDICATOR:", smm_col)
if not all([diag_col, icd_col, smm_col]):
    raise ValueError("Could not auto-detect all required columns. "
                     f"Found: DIAG={diag_col}, ICD={icd_col}, SMM={smm_col}. "
                     "Open the file, note the exact headers, and extend `*_candidates` above.")

Guessed columns → DIAGNOSIS: DIAGNOSIS_OR_PROCEDURE | ICD_10: ICD-10 | SMM_INDICATOR: SEVERE_MATERNAL_MORBIDITY_INDICATOR


In [40]:
smm_ref_std = smm_ref.rename(columns={
    diag_col: "DIAGNOSIS",
    icd_col:  "ICD_10",
    smm_col:  "SMM_INDICATOR"
})

In [41]:
smm_ref_dx = (
    smm_ref_std.assign(
        DIAGNOSIS=smm_ref_std["DIAGNOSIS"].astype(str).str.strip().str.upper(),
        ICD_10   =smm_ref_std["ICD_10"].astype(str).str.strip().str.upper(),
        SMM_INDICATOR_RAW=smm_ref_std["SMM_INDICATOR"].astype(str).str.strip()
    )
    .loc[lambda d: d["DIAGNOSIS"].eq("DX"), ["SMM_INDICATOR_RAW", "ICD_10"]]
    .drop_duplicates()
)

In [42]:
# =============== 2) Unpivot codes & descriptions and align by slot ===============
if "DIM_MEMBER_KEY" not in mm_raw.columns:
    raise ValueError("Main file must contain DIM_MEMBER_KEY")

code_cols_all = [f"DIAGNOSIS_CODE_{i}" for i in range(1, 11)]
desc_cols_all = [f"DIAGNOSIS_DESCRIPTION_SHORT_{i}" for i in range(1, 11)]

code_cols = [c for c in code_cols_all if c in mm_raw.columns]
desc_cols = [c for c in desc_cols_all if c in mm_raw.columns]

# Melt codes
codes_long = (
    mm_raw[["DIM_MEMBER_KEY"] + code_cols]
      .melt(id_vars=["DIM_MEMBER_KEY"], var_name="code_slot", value_name="icd_code")
      .assign(
          slot_id=lambda d: d["code_slot"].str.extract(r"(\d+)$").astype(int),
          icd_code_norm=lambda d: d["icd_code"].astype(str).str.strip().str.upper()
      )[["DIM_MEMBER_KEY","slot_id","icd_code_norm"]]
)

# Melt descriptions (if present)
if desc_cols:
    desc_long = (
        mm_raw[["DIM_MEMBER_KEY"] + desc_cols]
          .melt(id_vars=["DIM_MEMBER_KEY"], var_name="desc_slot", value_name="icd_desc")
          .assign(
              slot_id=lambda d: d["desc_slot"].str.extract(r"(\d+)$").astype(int),
              icd_desc_norm=lambda d: d["icd_desc"].astype(str).str.strip().str.lower()
          )[["DIM_MEMBER_KEY","slot_id","icd_desc_norm"]]
    )
else:
    desc_long = pd.DataFrame(columns=["DIM_MEMBER_KEY","slot_id","icd_desc_norm"])

# Align by member+slot
mm_codes_long = codes_long.merge(desc_long, on=["DIM_MEMBER_KEY","slot_id"], how="left")
mm_codes_long["icd_desc_norm"] = mm_codes_long["icd_desc_norm"].fillna("")

# =============== 3) Join to SMM reference to carry smm_indicator_raw ===============
smm_matches_join = mm_codes_long.merge(
    smm_ref_dx.rename(columns={"ICD_10": "icd_code_norm"}),
    on="icd_code_norm",
    how="left"
)

In [43]:
# =============== 4) Chronic condition flags (description OR ICD pattern) ===============
CC_TEXT_PATTERNS = {
    "diabetes":       r"diabet",
    "hypertension":   r"hypertens",
    "asthma":         r"\basthma\b",
    "copd":           r"\bcopd\b|chronic obstructive",
    "depression":     r"depress",
    "heart_failure":  r"heart failure|cardiac failure|congestive heart failure|\bchf\b",
    "ischemic":       r"ischemi|ischaemi|coronary artery|\bcad\b",
    "stroke":         r"\bstroke\b|cerebrovascular|\bcva\b|ischemic stroke|hemorrhagic stroke",
}
CC_ICD_PATTERNS = {
    "diabetes":      r"^E0[8-9]\.|^E1[0-3]\.",
    "hypertension":  r"^I1[0-6]\.",
    "asthma":        r"^J45\.",
    "copd":          r"^J44\.",
    "depression":    r"^F3[2-3]\.",
    "heart_failure": r"^I50\.|^I11\.0|^I13\.(0|2)",
    "ischemic":      r"^I2[0-5]\.",
    "stroke":        r"^I6[0-9]\.",
}

for nm, pat in CC_TEXT_PATTERNS.items():
    smm_matches_join[f"has_{nm}_txt"] = smm_matches_join["icd_desc_norm"].str.contains(pat, na=False, regex=True)

for nm, pat in CC_ICD_PATTERNS.items():
    smm_matches_join[f"has_{nm}_icd"] = smm_matches_join["icd_code_norm"].str.contains(pat, na=False, regex=True)

for nm in CC_TEXT_PATTERNS.keys():
    smm_matches_join[f"has_{nm}"] = (
        smm_matches_join[f"has_{nm}_txt"] | smm_matches_join[f"has_{nm}_icd"]
    ).astype(int)

smm_matches_join.drop(columns=[c for c in smm_matches_join.columns if c.endswith("_txt") or c.endswith("_icd")],
                      inplace=True)

# =============== 5) SMM indicator flags (from smm_indicator_raw) ===============
tmp = (
    smm_matches_join
      .assign(smm_indicator_raw=lambda d: d["SMM_INDICATOR_RAW"].fillna("").str.strip().str.lower())
      .loc[lambda d: d["smm_indicator_raw"] != "", ["DIM_MEMBER_KEY","smm_indicator_raw"]]
      .drop_duplicates()
)

if tmp.empty:
    smm_flags_wide = pd.DataFrame({"DIM_MEMBER_KEY": smm_matches_join["DIM_MEMBER_KEY"].unique()})
else:
    smm_flags_wide = (
        pd.crosstab(tmp["DIM_MEMBER_KEY"], tmp["smm_indicator_raw"])
          .rename(columns=lambda c: f"flag_smm_{re.sub(r'\\s+','_',c)}")
          .reset_index()
    )

/tmp/ipython-input-393759403.py:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  smm_matches_join[f"has_{nm}_icd"] = smm_matches_join["icd_code_norm"].str.contains(pat, na=False, regex=True)


In [44]:
# =============== 6) Collapse chronic flags to one row per member ===============
chronic_cols = [c for c in smm_matches_join.columns if c.startswith("has_")]
if chronic_cols:
    chronic_flags_wide = (
        smm_matches_join[["DIM_MEMBER_KEY"] + chronic_cols]
          .groupby("DIM_MEMBER_KEY", as_index=False)
          .max(numeric_only=True)
    )
else:
    chronic_flags_wide = pd.DataFrame({"DIM_MEMBER_KEY": smm_matches_join["DIM_MEMBER_KEY"].unique()})



In [45]:
# =============== 7) Keep demographics & combine into final wide ===============
demo_cols = ["DELIVERY_DATE","MEMBER_RACE","MEMBER_AGE","MEMBER_ZIP_CODE_4"]
demo_cols = ["DIM_MEMBER_KEY"] + [c for c in demo_cols if c in mm_raw.columns]

member_demo = (
    mm_raw[demo_cols]
      .replace({"": np.nan})
      .groupby("DIM_MEMBER_KEY", as_index=False)
      .agg({c: "first" for c in demo_cols if c != "DIM_MEMBER_KEY"})
)

final_wide = (
    member_demo
      .merge(chronic_flags_wide, on="DIM_MEMBER_KEY", how="left")
      .merge(smm_flags_wide,     on="DIM_MEMBER_KEY", how="left")
)

# Fill only flag columns (don’t clobber demo fields)
flag_cols = [c for c in final_wide.columns if c.startswith("has_") or c.startswith("flag_smm_")]
if flag_cols:
    final_wide[flag_cols] = final_wide[flag_cols].fillna(0).astype(int)

# =============== 8) Save / inspect ===============
final_wide.to_csv("member_flags_output_python.csv", index=False)
print("✅ Wrote member_flags_output_python.csv")
print("Shape:", final_wide.shape)
print("Columns:", list(final_wide.columns)[:12], "…")
final_wide.head()

✅ Wrote member_flags_output_python.csv
Shape: (25, 18)
Columns: ['DIM_MEMBER_KEY', 'DELIVERY_DATE', 'MEMBER_RACE', 'MEMBER_AGE', 'MEMBER_ZIP_CODE_4', 'has_diabetes', 'has_hypertension', 'has_asthma', 'has_copd', 'has_depression', 'has_heart_failure', 'has_ischemic'] …


,DIM_MEMBER_KEY,DELIVERY_DATE,MEMBER_RACE,MEMBER_AGE,MEMBER_ZIP_CODE_4,has_diabetes,has_hypertension,has_asthma,has_copd,has_depression,has_heart_failure,has_ischemic,has_stroke,flag_smm_acute myocardial infarction,flag_smm_acute renal failure,flag_smm_disseminated intravascular coagulation,flag_smm_puerperal cerebrovascular disorders,flag_smm_pulmonary edema / acute heart failure
0,M100108,9/24/2024,White,19,8726,1,1,1,1,1,1,1,1,1,1,1,1,1
1,M100189,2/26/2025,Black,41,9987,1,1,1,1,1,1,1,0,1,0,1,0,1
2,M100306,8/2/2024,Hispanic,22,9888,1,1,1,1,1,1,1,1,1,1,0,1,1
3,M100310,1/8/2025,Asian,23,5568,1,1,1,1,1,1,1,1,1,1,1,1,1
4,M100327,1/13/2024,Unknown,35,6270,1,0,1,1,1,1,1,1,1,0,0,1,1
